In [1]:
import numpy as np
from math import *
import pickle as pkl
from my_functions import *

### Machine parameters used for the computation of the theoretical expected growth

In [2]:
numBunches = 4
# Machine and beam parameters
betay = 73  # m in CC2, ~76 in CC1 (MAD-X)
Vcc = 1e6  # V
frev = 43.45e3  # Hz
Eb = 270e9  # eV
sigma_z = 0.138  # rms bunch length, m
beta_0 = 0.999999  # cmpt it from the rest
gamma_0 = 287.7  # cmt it
clight = 299792458  # light speed in meters/second
f_RF = 400.789e6  # CC frequency in Hz

### Measured phase and amplitude noise
The order of the arrays is the following:

[Coast1-Setting1, Coast1-Setting2, Coast2-Setting1, Coast2-Setting2, Coast2-Setting2, Coast3-Setting1, Coast3-Setting2, Coast3-Setting3]


In [11]:
L_PN_dB = np.array([-122.75, -101.48, -115.22, -111.28, -111.03, -106.46, -101.48])
sigma_L_PN_dB = np.array([0.47, 0.83, 0.61, 0.6, 0.83, 0.3, 0.76]) # uncertainty in the computation of the PN from the measured spectrum, averaged values over 1kHz
L_AN_dB = np.array([-128.15, -115.21, -124.06, -115.71, -116.92, -112.73, -106.99])
sigma_L_AN_dB = np.array([0.57, 0.66, 0.5, 0.4, 0.37, 0.3, 0.42 ])

### Load the effective PN (computed in another sctipr)

In [12]:
effective_PN_dict = pkl.load(open('effective_PN_4bunches_MD5.pkl', 'rb'))

### Uncertainties of the measured PN and AN when converted in rad^2/Hz, eq.10 and eq.11

In [13]:
sigma_S_PN_rad = np.log(10)*ssb_2_dsb(L_PN_dB)*sigma_L_PN_dB/5
sigma_S_AN_rad = np.log(10)*ssb_2_dsb(L_AN_dB)*sigma_L_AN_dB/5

### Load the bunch lengths at the beggining of each setting

In [14]:
# Load the initial bunch lengths for each coast settings
b = pkl.load(open('initial_bunch_lengths_4sigmat_seconds.pkl', 'rb'))
keys_b = list(b.keys())
print(keys_b)

['bunch_1', 'bunch_2', 'bunch_3', 'bunch_4']


#### covnert bunch length from ns to rad

In [15]:
sigma_phi_dict = {}
for key in keys_b: # iterate over the 4 bunches
    sigma_t = np.array(b[key])/4 # 1 sigma_t in seconds to be used in the function that converts to m
    sigma_z = bunch_length_time_to_m(sigma_t, clight)
    sigma_phi = bunch_length_m_to_rad(sigma_z, clight, f_RF)  # rms bunch length in rad
    sigma_phi_dict[key] = sigma_phi

### Compute the theoretical expected growth rate for every noise setting
- Different for each bunch due to different bunch lengths

In [19]:
sigma_dey_total_dict = {}
for key in keys_b:
    myC_PN = cmpt_bunch_length_correction_factor(sigma_phi_dict[key], 'PN')
    myC_AN = cmpt_bunch_length_correction_factor(sigma_phi_dict[key], 'AN')
    C_PN = betay*(Vcc*frev/(2*Eb))**2*myC_PN
    C_AN = betay * (Vcc * frev / (2 * Eb)) ** 2 * myC_AN

    sigma_dey_PN = C_PN*sigma_S_PN_rad # σ(dey/dt)_PN, eq.14
    sigma_dey_AN = C_AN * sigma_S_AN_rad #σ(dey/dt)_PN, eq.15

    sigma_dey_total_dict[key] = np.sqrt(sigma_dey_PN**2+sigma_dey_AN**2) # eq.17

print('Uncertainty on the theoretical expected growths due to the uncertainty of the noise in rad^2/Hz {}'.format(sigma_dey_total_dict[key]))


Uncertainty on the theoretical expected growths due to the uncertainty of the noise in rad^2/Hz [6.09212125e-14 1.33980136e-11 4.30221744e-13 1.02076677e-12
 1.52259864e-12 1.55128882e-12 1.22732682e-11]


### Compute the uncertainty of the effective noise, eq.19

In [21]:
# uncertainty on the effective noise in rad^2/Hz
sigma_S_PN_rad__eff_dict ={}
for key in keys_b:
    myC_PN = cmpt_bunch_length_correction_factor(sigma_phi_dict[key], 'PN')
    myC_AN = cmpt_bunch_length_correction_factor(sigma_phi_dict[key], 'AN')

    C_PN = betay * (Vcc * frev / (2 * Eb)) ** 2 * myC_PN
    C_AN = betay * (Vcc * frev / (2 * Eb)) ** 2 * myC_AN

    sigma_dey_PN = C_PN * sigma_S_PN_rad
    sigma_dey_AN = C_AN * sigma_S_AN_rad

    sigma_S_PN_rad__eff_dict[key] = np.sqrt(sigma_dey_PN**2+sigma_dey_AN**2)/C_PN

print(sigma_S_PN_rad__eff_dict[key])

[2.32052056e-13 5.43758050e-11 1.69068379e-12 4.14046654e-12
 6.03810110e-12 6.27919611e-12 4.99115216e-11]


### uncertainty on the effective noise in dbc/Hz, eq.22

In [22]:
sigma_L_PN_db__eff_dict = {}

for key in keys_b:
    sigma_L_PN_db__eff_dict[key] = np.log10(e)*sigma_S_PN_rad__eff_dict[key]*10/ssb_2_dsb(np.array(effective_PN_dict[key])) # not sure for x10, maybe x5

print('Uncertainty for bunch 1 on the effective PN in dBc/Hz {}'.format(sigma_L_PN_db__eff_dict['bunch_1']))


Uncertainty for bunch 1 on the effective PN in dBc/Hz [0.77411334 1.60121669 1.10288431 0.91765618 1.37061347 0.50113722
 1.22274528]
